In [1]:
from itertools import combinations

import os
import sys
from pathlib import Path

sys.path.append(str((Path(os.path.abspath("")) / "..").resolve()))
from sudoku_variants import Sudoku, SudokuAI, SudokuConst
from sudoku_variants.rule import Orthogonal, SubBoard, Knight, King, Consecutive
from sudoku_variants.func import rules as R
from sudoku_variants.helper.profile import profile


def standard_rules():
    return [Orthogonal(), SubBoard()]


def variant_rules():
    return [Knight(), King(), Consecutive()]

### AI

In [2]:
@profile(10)
def run(rules):
    board = [[0 for _ in range(SudokuConst.NUM_COL)] for _ in range(SudokuConst.NUM_ROW)]
    sudoku = Sudoku(board, rules)
    SudokuAI.solve(sudoku)


for length in range(len(variant_rules()) + 1):
    for subset in combinations(variant_rules(), length):
        print(R.to_name(list(subset)))
        rules = standard_rules() + list(subset)
        run(rules)


Time: 0.0359s, 10x | 0.0036s/time
Knight
Time: 2.7705s, 10x | 0.2770s/time
King
Time: 0.0691s, 10x | 0.0069s/time
Consecutive
Time: 0.0312s, 10x | 0.0031s/time
Knight, King
Time: 2.1659s, 10x | 0.2166s/time
Knight, Consecutive
Time: 1.3983s, 10x | 0.1398s/time
King, Consecutive
Time: 0.0778s, 10x | 0.0078s/time
Knight, King, Consecutive
Time: 2.1763s, 10x | 0.2176s/time


In [8]:
@profile(10)
def run(rules, to_erase):
    SudokuAI.generate(rules, max_erased=to_erase)


for to_erase in [35, 40, 50, 60]:
    print(to_erase)
    rules = [Orthogonal(), SubBoard()]
    run(rules, to_erase)

35
Time: 5.7043s, 10x | 0.5704s/time
40
Time: 24.9208s, 10x | 2.4921s/time
50
Time: 471.9391s, 10x | 47.1939s/time
60


In [3]:
@profile(1)
def run():
    sudoku = SudokuAI.generate([Orthogonal(), SubBoard(), Knight(), King(), Consecutive()], max_erased=40)
    print(sudoku)


run()

KeyboardInterrupt: 